In [1]:
import pandas as pd

In [2]:
# ai_result_path = "./AI_Mapping_Result/V3/1_BAY_mapping_result_V3_rev6.csv"
ai_result_path = "./AI_Mapping_Result/V4/1_BAY_mapping_result_V4_rev1.csv"
prod_result_path = "./Listed_Mapping_Prod/2024/BAY_Q2_2024.csv"

In [3]:
th_month = [
    'มกราคม', 
    'กุมภาพันธ์', 
    'มีนาคม',
    'เมษายน',
    'พฤษภาคม',
    'มิถุนายน',
    'กรกฎาคม',
    'สิงหาคม',
    'กันยายน',
    'ตุลาคม',
    'พฤศจิกายน',
    'ธันวาคม',
]

th_month_short = [
    'ม.ค.', 
    'ก.พ.', 
    'มี.ค.',
    'เม.ย.',
    'พ.ค.',
    'มิ.ย.',
    'ก.ค.',
    'ส.ค.',
    'ก.ย.',
    'ต.ค.',
    'พ.ย.',
    'ธ.ค.',
]

In [10]:
def replace_comma(name):
    return name.replace(",", "")

def replace_month(name):
    for month in th_month:
        if (month in name):
            name = name.replace(month, "")

    for month in th_month_short:
        if (month in name):
            name = name.replace(month, "")
    return name

In [11]:
dtype = {
    'query': 'string',
    'set_code': 'string'
}

ai_df = pd.read_csv(ai_result_path, encoding='utf-8', dtype=dtype)

ai_df_query = ai_df['query']
ai_df_query.replace(to_replace=' ', value='', regex=True, inplace=True)
ai_df_query.replace(to_replace=r'[0-9]', value='', regex=True, inplace=True)
ai_df['query'] = ai_df_query

ai_df_query = ai_df['query']
for index, row in ai_df_query.items():
    if (pd.isna(row)):
        continue
    name = row
    name_replace = replace_month(name)
    name_replace = replace_comma(name_replace)
    ai_df['query'].iloc[index] = name_replace

ai_df.iloc[20]['query']

'ทุนเรือนหุ้นทุนจดทะเบียนหุ้นสามัญหุ้นมูลค่าหุ้นละบาท'

In [16]:
dtype = {
    'account_name': 'string',
    'account_code': 'string',
    'parent_account_code': 'string',
}

prod_df = pd.read_csv(prod_result_path, encoding='utf-8', dtype=dtype)
prod_acc_name = prod_df['account_name']
prod_acc_name.replace(to_replace=r'\{{\w+\}}', value='', regex=True, inplace=True)
prod_df['account_name'] = prod_acc_name
prod_df.iloc[22]

account_name                      ทุนจดทะเบียนหุ้นสามัญหุ้นมูลค่าหุ้นละบาท
taxonomy_account_name    ทุนจดทะเบียนหุ้นสามัญ{{number}}หุ้นมูลค่าหุ้นล...
account_code                                                        310120
parent_account_code                                                 310100
values                                        ['75,741,437', '75,741,437']
Name: 22, dtype: object

In [20]:
compared_list = []

for index, row in prod_df.iterrows():
    prod_taxo_name = row['taxonomy_account_name']
    prod_acc_code = row['account_code']
    prod_parent_acc_code = row['parent_account_code']
    prod_acc_name = row['account_name']
    prod_values = row['values']
    # print(prod_acc_name)
    
    ai_row = ai_df[(ai_df['set_code'] == prod_acc_code) | (ai_df['set_code'] == prod_parent_acc_code)]
    # ai_row = ai_df[(ai_df['query'] == prod_acc_name) & ((ai_df['set_code'] == prod_acc_code) | (ai_df['set_code'] == prod_parent_acc_code))]
    # print(len(ai_row.index))

    select_index = None
    for i, r in ai_row.iterrows():
        if (prod_acc_name in r['query']):
            select_index = i
            break
    
    # print(select_index)
    # if (len(ai_row.index) > 1):
    #     print(prod_acc_name)
    #     print(len(ai_row.index))
    #     print()

    ai_acc_code = ""
    ai_acc_name = ""
    if (len(ai_row.index) > 0 and select_index is not None):
        ai_acc_code = ai_df['set_code'].iloc[select_index]
        ai_acc_name = ai_df['query'].iloc[select_index]
    else:
        print(f"Not found {prod_acc_code} in AI mapping")
    

    compare_item = {
        "taxonomy_account_name": prod_taxo_name,
        "ai_account_code": ai_acc_code,
        "ai_acc_name": ai_acc_name,
        "prod_account_name": prod_acc_name,
        "prod_account_code": prod_acc_code,
        "prod_parent_account_code": prod_parent_acc_code,
        "is_match": str(ai_acc_code) == str(prod_acc_code),
        "is_match_parent": str(ai_acc_code) == str(prod_parent_acc_code)
    }

    # print(compare_item)
    # print(compare_item['taxonomy_account_name'], compare_item['ai_account_code'], compare_item['ai_acc_name'], compare_item['prod_account_name'], compare_item['prod_account_code'], compare_item['is_match'])
    compared_list.append(compare_item)


Not found 209999 in AI mapping
Not found 309999 in AI mapping
Not found 430700 in AI mapping
Not found 430700 in AI mapping
Not found 510400 in AI mapping
Not found 510800 in AI mapping
Not found 512300 in AI mapping
Not found 514800 in AI mapping
Not found 512000 in AI mapping
Not found 514710 in AI mapping
Not found 518300 in AI mapping


In [22]:
count_unmatch = 0
count_match_parent = 0
for item in compared_list:
    if (item['is_match'] == False):
        if (item['is_match_parent'] == True):
            count_match_parent += 1
            print("Match Parent")
        if (item['ai_account_code'] == "" and item['ai_acc_name'] == ""):
            print("Not Match accode or name with AI")
        count_unmatch += 1
        print(item)
        print("\n")

print(f"Total Unmatch: {count_unmatch}")

Match Parent
{'taxonomy_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'ai_account_code': '100600', 'ai_acc_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'prod_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'prod_account_code': '100690', 'prod_parent_account_code': '100600', 'is_match': False, 'is_match_parent': True}


Match Parent
{'taxonomy_account_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'ai_account_code': '202000', 'ai_acc_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'prod_account_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'prod_account_code': '202090', 'prod_parent_account_code': '202000', 'is_match': False, 'is_match_parent': True}


Not Match accode or name with AI
{'taxonomy_account_name': 'รวมหนี้สิน', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'รวมหนี้สิน', 'prod_account_code': '209999', 'prod_parent_account_code': '309999', 'is_match': False, 'is_match_pa

In [19]:
no_code_count = 0
for item in compared_list:
    if (item['is_match'] == False):
        if (item['ai_account_code'] == ''):
            no_code_count += 1
            print(item['prod_account_code'])

print(f"No Code From AI: {no_code_count}")

209999
309999
430700
430700
510400
510800
512300
514800
512000
514710
518300
No Code From AI: 11
